# 🔍Sanity Check

This notebook verifies that the Dev Container (Python 3.13 + CUDA 13) can access the GPU through PyTorch and perform GPU accelerated computations via RAPIDS.

### GPU, CUDA, RAPIDS Sanity Check

In [2]:
# must be BEFORE importing pandas
%load_ext cudf.pandas

In [3]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %code_wrap  %colors  %conda  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %mamba  %man  %matplotlib  %micromamba  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%code_wrap  %%cudf.pandas.line_profile  %%cudf.pandas.prof

In [ ]:
import IPython
import pandas as pd
import cudf
import cudf.pandas
import cupy
import torch
import platform

print("Python:                  ", platform.python_version())
print("IPython:                 ", IPython.__version__)
print("Magics present:          ","cudf.pandas.profile" in get_ipython().magics_manager.magics['cell'])
print()

if torch.cuda.is_available():
    print("GPU:                     ", torch.cuda.get_device_name(0))
    print("CUDA runtime version:    ", torch.version.cuda)
    print("cuDF:                    ", cudf.__version__)
    print("cudf.pandas installed:   ", hasattr(cudf.pandas, "install"))
    print("cupy config:             ", cupy.show_config())

    print()

    print("PyTorch:                 ", torch.__version__)
    print("CUDA available (torch):  ", torch.cuda.is_available())

Python:                   3.11.10
IPython:                  8.27.0
Magics present:           True

GPU:                      NVIDIA GeForce RTX 4090 Laptop GPU
CUDA runtime version:     12.0
cuDF:                     24.08.03
cudf.pandas installed:    True

PyTorch:                  2.4.0.post301
CUDA available (torch):   True


In [17]:
%%cudf.pandas.profile on

# cuDF accelerated data frame
df = cudf.DataFrame({"a": range(1_000_000)})
print("sum (cuDF):", int(df["a"].sum()))

%%cudf.pandas.profile off

sum (cuDF): 499999500000


UsageError: Line magic function `%%cudf.pandas.profile` not found.


                                                                                              
                                  Total time elapsed: 0.141 seconds                           
                                0 GPU function calls in 0.000 seconds                         
                                0 CPU function calls in 0.000 seconds                         
                                                                                              
                                                Stats                                         
                                                                                              
┏━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Function ┃ GPU ncalls ┃ GPU cumtime ┃ GPU percall ┃ CPU ncalls ┃ CPU cumtime ┃ CPU percall ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
└──────────┴────────────┴─────────────┴─────────────┴────────────┴─────────────┴─────────────┘

In [18]:
%%cudf.pandas.profile on

# cuDF implicit via pandas-accelerated
df = pd.DataFrame({"a": range(1_000_000)})
print("sum (pandas-accelerated):", df["a"].sum())

%%cudf.pandas.profile off

sum (pandas-accelerated): 499999500000


UsageError: Line magic function `%%cudf.pandas.profile` not found.


                                                                                                           
                                         Total time elapsed: 0.141 seconds                                 
                                       3 GPU function calls in 0.011 seconds                               
                                       0 CPU function calls in 0.000 seconds                               
                                                                                                           
                                                       Stats                                               
                                                                                                           
┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Function              ┃ GPU ncalls ┃ GPU cumtime ┃ GPU percall ┃ CPU ncalls ┃ CPU cumtime ┃ CPU percall ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ DataFrame             │ 1          │ 0.010       │ 0.010       │ 0          │ 0.000       │ 0.000       │
│ DataFrame.__getitem__ │ 1          │ 0.001       │ 0.001       │ 0          │ 0.000       │ 0.000       │
│ Series.sum            │ 1          │ 0.001       │ 0.001       │ 0          │ 0.000       │ 0.000       │
└───────────────────────┴────────────┴─────────────┴─────────────┴────────────┴─────────────┴─────────────┘

In [19]:
%%cudf.pandas.profile on

# cuML logistic regression
from cuml.linear_model import LogisticRegression
from cuml.model_selection import train_test_split

X = cupy.random.rand(200_000, 20)
y = (X.sum(axis=1) > 10).astype(cupy.int32)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
clf = LogisticRegression(max_iter=100)
clf.fit(X_train, y_train)
acc = float((clf.predict(X_test) == y_test).mean())

print("cuML GPU accuracy:", acc)

%%cudf.pandas.profile off

cuML GPU accuracy: 0.999275


UsageError: Line magic function `%%cudf.pandas.profile` not found.


                                                                                              
                                  Total time elapsed: 0.199 seconds                           
                                0 GPU function calls in 0.000 seconds                         
                                0 CPU function calls in 0.000 seconds                         
                                                                                              
                                                Stats                                         
                                                                                              
┏━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Function ┃ GPU ncalls ┃ GPU cumtime ┃ GPU percall ┃ CPU ncalls ┃ CPU cumtime ┃ CPU percall ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
└──────────┴────────────┴─────────────┴─────────────┴────────────┴─────────────┴─────────────┘

In [20]:
%%cudf.pandas.profile on

# cuDF's pandas accelerator for pandas
import pandas as pd

print("pandas version:", pd.__version__)
df = pd.DataFrame({"x": range(10_000_000)})
_ = df.groupby(pd.cut(df["x"], 100)).x.mean()

%cudf.pandas.profile off

: 

In [1]:
%%cudf.pandas.profile on

# cuML's scikit-learn accelerator
from cuml.linear_model import LogisticRegression
from cuml.model_selection import train_test_split
import cupy as cp

X = cp.random.rand(200_000, 20)
y = (X.sum(axis=1) > 10).astype(cp.int32)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
clf = LogisticRegression(max_iter=100)
clf.fit(X_train, y_train)
print("GPU accuracy:", float((clf.predict(X_test) == y_test).mean()))

%%cudf.pandas.profile off

UsageError: Cell magic `%%cudf.pandas.profile` not found.


### Torch Sanity Check

In [3]:
# Run a small GPU tensor operation to confirm GPU compute works
if torch.cuda.is_available():
    x = torch.rand((10000, 10000), device="cuda")
    y = torch.rand((10000, 10000), device="cuda")
    z = torch.mm(x, y)
    print("✅ GPU matrix multiplication completed successfully.")
    del x, y, z
else:
    print("⚠️  Skipping GPU compute test (CUDA not available).")

✅ GPU matrix multiplication completed successfully.
